### ЛАБОРАТОРНА РОБОТА 4

#### 1. Вибір задачі та датасету

Використаємо датасет та модель з лабораторної №2

In [1]:
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, pipeline

model_name = "../lab2/models/mt5_fr2en_copy/checkpoint-500"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

generator = pipeline("text2text-generation", model=model.eval(), tokenizer=tokenizer)

/home/yaroslavp/workspace/master_IASA/nlp/lab4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
Device set to use cuda:0


In [2]:
text = 'Bonjour comment allez-vous?'

outputs = generator(
    "translate French to English: " + text,
    max_length=128,
    do_sample=True,
    temperature=0.4
)
outputs

Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Hello, how are you doing?'}]

In [3]:
outputs[0]['generated_text']

'Hello, how are you doing?'

In [3]:
from datasets import load_dataset
from gptqmodel import GPTQModel, QuantizeConfig

model_id = model_name
quant_path = "api/service/model/mt5_fr2en-gptqmodel-4bit"

calibration_dataset = load_dataset(
    "allenai/c4",
    data_files="en/c4-train.00001-of-01024.json.gz",
    split="train"
  ).select(range(1024))["text"]

quant_config = QuantizeConfig(bits=4, group_size=128)

model = GPTQModel.load(model_id, quant_config)

# increase `batch_size` to match gpu/vram specs to speed up quantization
model.quantize(calibration_dataset, batch_size=1)

model.save(quant_path)

ModuleNotFoundError: No module named 'datasets'

In [2]:
import torch
print(torch.version.cuda)         # e.g., '11.8'
print(torch.cuda.is_available())

12.6
True
